# Advanced Generative Chatbot Using BERT model
## AAI-520 Team 3 Final Project

Team 3 Members:  Tyler Foreman, Ahmed Ahmed, Tursun Alkam

Date:  October 23, 2023

GitHub Repository: https://github.com/t4ai/aai520-team3-final

In [1]:
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import random
import statistics
import spacy
import nltk
import os
import json

#from transformers import BertModel, BertTokenizerFast, AdamW

## Load Dataset and Process into Format for BERT Tokenizer
 
 - For Dev and Training:
 - Load data and process into contexts, questions and answers { text, start, end}
 --  For each question/answer: add context entry to contexts[], add question to questions[] and answer to answers[]

In [4]:
# helper function to load dataset from JSON file
def load_squad_data_file(file_path):
    f = open(file_path, 'rb')
    squad_data = json.load(f)

    contexts = []
    questions = []
    answers = []

    # traverse JSON object tree
    for json_groups in squad_data['data']:
        for paragraph in json_groups['paragraphs']:
            curr_context = paragraph['context']
            for qa in paragraph['qas']:
                curr_question = qa['question']
                for answer in qa['answers']:
                    # add entries for each answer to all lists
                    contexts.append(curr_context.lower())
                    questions.append(curr_question.lower())
                    answers.append(answer)
                    
    return contexts, questions, answers

In [11]:
# load the datasets from JSON
contexts_train, questions_train, answers_train = load_squad_data_file('./dataset/train-v1.1.json')
contexts_test, questions_test, answers_test = load_squad_data_file('./dataset/dev-v1.1.json')

In [18]:
# inspect some samples to make sure everything was loaded correctly
print("[Train Samples]")
print(contexts_train[0])
print(questions_train[:5])
print(answers_train[:5])


print("\n[Test Samples]")
print(contexts_test[0])
print(questions_test[:5])
print(answers_test[:5])

[Train Samples]
architecturally, the school has a catholic character. atop the main building's gold dome is a golden statue of the virgin mary. immediately in front of the main building and facing it, is a copper statue of christ with arms upraised with the legend "venite ad me omnes". next to the main building is the basilica of the sacred heart. immediately behind the basilica is the grotto, a marian place of prayer and reflection. it is a replica of the grotto at lourdes, france where the virgin mary reputedly appeared to saint bernadette soubirous in 1858. at the end of the main drive (and in a direct line that connects through 3 statues and the gold dome), is a simple, modern stone statue of mary.
['to whom did the virgin mary allegedly appear in 1858 in lourdes france?', 'what is in front of the notre dame main building?', 'the basilica of the sacred heart at notre dame is beside to which structure?', 'what is the grotto at notre dame?', 'what sits on top of the main building at 

## Data Cleanup
-  Perform routine cleanup on data:
    - remove punctuation marks
    - convert to lowercase
    - remove numbers
    - remove whitespace
- Remove stop words
- Lemmatize the text
    - Normalize to base words